<a href="https://colab.research.google.com/github/ilyandho/FPL-Optimal-Transfer/blob/main/FPL_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [21]:
%pip install sklearn


  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-pypi-packag

## Fetch and extract important attributes from FPL API

Store the data to _`player_data.csv`_


In [22]:
import requests
import json
import pandas as pd

base_url = 'https://fantasy.premierleague.com/api/'

# This functionn fetches the player details using FPL API up to a given week


def get_player_data(gw):
    # Get the general, inorder to have access to the team ids in the elements component
    general_info = requests.get(base_url + "/bootstrap-static/").json()

    # player_history will contain the data
    player_history = []

    # For each game week, the stats for all players
    for gwk in range(1, gw+1):
        r = requests.get(base_url + 'event/' + str(gwk) + '/live/').json()
        # Get the fixtures for each game week gwk.
        # It will contain 10 matches with each match containing the stats for the home and away team difficulty rating.
        fixture = requests.get(base_url + 'fixtures?event=' + str(gwk)).json()

        # For each player we will also need the match difficulty for both player's team and plyaer's opponent team.
        for player in r['elements']:

            team_h_difficulty = 0
            team_a_difficulty = 0

            # Get the player element id in the player component and compare it to the id of the element in elements component of general_info
            # Also get the player position which will later be used to filter the data by position
            player_team = None
            position = None
            for element in general_info['elements']:
                if player['id'] == element['id']:
                    player_team = element['team']
                    position = element['element_type']

            # For each match in the fixture array, we compare the player team id with the away team and home team ids in the match.
            # If they match either, then update team_h_difficulty and team_a_difficulty
            for match_ in fixture:
                if player_team == match_['team_a'] or player_team == match_['team_h']:
                    team_h_difficulty = match_['team_h_difficulty']
                    team_a_difficulty = match_['team_a_difficulty']

            # Create a temporary variable containing the gwk, team_a_difficulty, team_h_difficulty and position
            temp_data = {'id': int(player['id']), 'gw': gwk, 'team_h_difficulty': team_h_difficulty,
                         'team_a_difficulty': team_a_difficulty, 'position': position}

            # The stats element in each player component contains:
            #  - minutes, goals_scored, assists, clean_sheets, goals_conceded, own_goals, penalties_saved,
            #  - penalties_missed, yellow_cards, red_cards, saves, bonus, bps, influence,
            #  - creativity, threat, ict_index, starts, expected_goals, expected_assists,
            #  - expected_goal_involvements, expected_goals_conceded, total_points, in_dreamteam
            for key in player['stats'].keys():
                temp_data.update({key: player['stats'][key]})

            # Append the player info to the player_history array
            player_history.append(temp_data)
    return player_history


player_data = pd.DataFrame(get_player_data(30))

player_data

,id,gw,team_h_difficulty,team_a_difficulty,position,minutes,goals_scored,assists,clean_sheets,goals_conceded,...,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,total_points,in_dreamteam
0,1,1,2,5,4,0,0,0,0,0,...,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,0,False
1,2,1,2,5,2,0,0,0,0,0,...,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,0,False
2,3,1,2,5,3,0,0,0,0,0,...,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,0,False
3,4,1,2,5,3,0,0,0,0,0,...,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,0,False
4,5,1,2,5,2,4,0,0,0,0,...,0.0,0.0,0.0,0,0.00,0.00,0.00,0.02,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22833,839,30,2,4,3,0,0,0,0,0,...,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,0,False
22834,840,30,2,4,4,0,0,0,0,0,...,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,0,False
22835,841,30,2,2,3,0,0,0,0,0,...,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,0,False
22836,842,30,2,2,3,0,0,0,0,0,...,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,0,False


In [23]:
player_data.columns

Index(['id', 'gw', 'team_h_difficulty', 'team_a_difficulty', 'position',
       'minutes', 'goals_scored', 'assists', 'clean_sheets', 'goals_conceded',
       'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards',
       'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity',
       'threat', 'ict_index', 'starts', 'expected_goals', 'expected_assists',
       'expected_goal_involvements', 'expected_goals_conceded', 'total_points',
       'in_dreamteam'],
      dtype='object')

In [24]:
# Store data to csv
player_data.to_csv('./content/player_data.csv', encoding='utf-8-sig', )

## Fetch stored player data in csv


In [25]:
import pandas as pd
# Load the player detaills
df = pd.read_csv('./content/player_data.csv')
df.tail()

,Unnamed: 0,id,gw,team_h_difficulty,team_a_difficulty,position,minutes,goals_scored,assists,clean_sheets,...,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,total_points,in_dreamteam
22833,22833,839,30,2,4,3,0,0,0,0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,False
22834,22834,840,30,2,4,4,0,0,0,0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,False
22835,22835,841,30,2,2,3,0,0,0,0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,False
22836,22836,842,30,2,2,3,0,0,0,0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,False
22837,22837,843,30,2,5,3,0,0,0,0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,False


In [26]:
# Get the data statics
df.describe()

,Unnamed: 0,id,gw,team_h_difficulty,team_a_difficulty,position,minutes,goals_scored,assists,clean_sheets,...,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,total_points
count,22838.00000,22838.000000,22838.000000,22838.000000,22838.000000,22838.000000,22838.000000,22838.000000,22838.000000,22838.000000,...,22838.000000,22838.000000,22838.000000,22838.000000,22838.000000,22838.000000,22838.000000,22838.000000,22838.000000,22838.000000
mean,11418.50000,383.021806,16.069884,2.513618,2.916367,2.575401,25.265216,0.039977,0.036124,0.063053,...,5.991208,4.052855,3.798713,1.383619,0.282249,0.040455,0.025396,0.065842,0.443002,1.046677
std,6592.90706,222.936607,8.636192,1.048487,1.233877,0.856158,38.149475,0.217718,0.205797,0.243063,...,12.369450,10.516684,10.739667,2.793724,0.456959,0.150496,0.083918,0.192029,0.813138,2.287807
min,0.00000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-4.000000
25%,5709.25000,191.000000,9.000000,2.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,11418.50000,381.000000,16.000000,2.000000,3.000000,3.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,17127.75000,571.000000,24.000000,3.000000,4.000000,3.000000,65.000000,0.000000,0.000000,0.000000,...,6.400000,1.200000,0.000000,1.800000,1.000000,0.000000,0.010000,0.010000,0.630000,1.000000
max,22837.00000,843.000000,30.000000,5.000000,5.000000,4.000000,180.000000,3.000000,4.000000,1.000000,...,130.000000,181.800000,142.000000,27.400000,2.000000,2.770000,1.470000,3.880000,7.820000,23.000000


In [27]:
# Get details about each feature
# From the data info, all features have numerical values except the 'in_dreamteam' which has a boolean value
# No observation (row) for each feature (column) is null
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22838 entries, 0 to 22837
Data columns (total 30 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  22838 non-null  int64  
 1   id                          22838 non-null  int64  
 2   gw                          22838 non-null  int64  
 3   team_h_difficulty           22838 non-null  int64  
 4   team_a_difficulty           22838 non-null  int64  
 5   position                    22838 non-null  int64  
 6   minutes                     22838 non-null  int64  
 7   goals_scored                22838 non-null  int64  
 8   assists                     22838 non-null  int64  
 9   clean_sheets                22838 non-null  int64  
 10  goals_conceded              22838 non-null  int64  
 11  own_goals                   22838 non-null  int64  
 12  penalties_saved             22838 non-null  int64  
 13  penalties_missed            228

In [28]:
# Of the features, we want to use features that are available at the time the model is running.
# This makes sure that the model only depends on games stats that are available before the match starts.
# These will store in the 'attributes' variable and we will subsquently be using these for the rest of the work.
attributes = ['team_h_difficulty', 'team_a_difficulty', 'position',
              'threat', 'ict_index', 'expected_goals', 'expected_assists',
              'expected_goal_involvements', 'expected_goals_conceded', 'total_points']
players_data = df[attributes]
players_data

,team_h_difficulty,team_a_difficulty,position,threat,ict_index,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,total_points
0,2,5,4,0.0,0.0,0.0,0.0,0.0,0.00,0
1,2,5,2,0.0,0.0,0.0,0.0,0.0,0.00,0
2,2,5,3,0.0,0.0,0.0,0.0,0.0,0.00,0
3,2,5,3,0.0,0.0,0.0,0.0,0.0,0.00,0
4,2,5,2,0.0,0.0,0.0,0.0,0.0,0.02,1
...,...,...,...,...,...,...,...,...,...,...
22833,2,4,3,0.0,0.0,0.0,0.0,0.0,0.00,0
22834,2,4,4,0.0,0.0,0.0,0.0,0.0,0.00,0
22835,2,2,3,0.0,0.0,0.0,0.0,0.0,0.00,0
22836,2,2,3,0.0,0.0,0.0,0.0,0.0,0.00,0


In [29]:
# Group the players by position
# Here we will train models for each position i.e each position will have its own model for predicting the data
goalkeepers = players_data.loc[players_data['position'] == 1]
defenders = players_data.loc[players_data['position'] == 2]
midfielders = players_data.loc[players_data['position'] == 3]
forwards = players_data.loc[players_data['position'] == 4]

forwards

,team_h_difficulty,team_a_difficulty,position,threat,ict_index,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,total_points
0,2,5,4,0.0,0.0,0.00,0.00,0.00,0.00,0
7,2,5,4,0.0,0.0,0.00,0.00,0.00,0.00,0
12,2,5,4,36.0,7.2,0.21,0.01,0.22,0.30,8
32,2,2,4,0.0,0.0,0.00,0.00,0.00,0.00,0
38,3,4,4,0.0,0.0,0.00,0.00,0.00,0.00,0
...,...,...,...,...,...,...,...,...,...,...
22815,2,2,4,0.0,0.0,0.00,0.00,0.00,0.00,0
22816,2,2,4,0.0,0.0,0.00,0.00,0.00,0.00,0
22818,2,5,4,0.0,0.0,0.00,0.00,0.00,0.00,0
22830,2,4,4,7.0,1.3,0.04,0.00,0.04,0.97,2


In [30]:
from sklearn.model_selection import train_test_split

# define a function that splits and returns features_train, features_test, target_train, target_test


def split_position_data(data):
    # Store the 'total_points' target in the 'player_target' variable
    # and the rest in the player_features variable
    player_target = data['total_points']
    player_features = data.drop("total_points", axis=1)

    # The train_test_split function splits the set into train and test sets while maintain the same data distribution over both sets.
    # It takes the feature and target sets and reutrns the respective train and test sets
    features_train, features_test, target_train, target_test = train_test_split(
        player_features, player_target, test_size=0.2)

    return {'feature_train': features_train, 'features_test': features_test, 'target_train': target_train, 'target_test': target_test}

In [31]:
# Split positions into the training and the test sets
goalkeepers_splits = split_position_data(goalkeepers)
defenders_splits = split_position_data(defenders)
midfielders_splits = split_position_data(midfielders)
forwards_splits = split_position_data(forwards)

## Using ML Models to Predict Points in Fantasy Premier League

For this project we are going to compare to Linear Regression, Decision Trees and Random Forests and find out how they all perform.


#### Helper functions


In [71]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

# For the linear model


def Linear_regression(features_train, features_test, target_train, target_test):
    # Before using our data, we need to do feature scaling and we opt for the 'standardization' method of scaling.
    # The 'standardization' is avaliable thorugh the StandardScaler() method
    # Transformers help in batching tasks in a pipepline. In this case, the data is scaled and then a linear regression model is fitted on the scaled data.
    # We use a transformer that takes the regression model and the transformation method
    # The TransformedTargetRegressor does the transformation and when we do the prediction, it automatically does the inverse transformation (scaling) and returns the values
    model = TransformedTargetRegressor(
        LinearRegression(), transformer=StandardScaler())

    # fit the transofrmer on the train data
    model.fit(features_train, target_train)

    # With the model fitted, we can predict the total_points given the feature_train and feature_test set
    pred_train = model.predict(features_train)
    pred_test = model.predict(features_test)

    # Evaluate the performance of the model on both sets using the root mean square error
    train_MSE = mean_squared_error(target_train, pred_train)
    test_MSE = mean_squared_error(target_test, pred_test)

    # Get the score of the model or the coeeficient of determination i.e how much of the target value can be explained by the model.
    # In this case, 0.6 implies that 60% of the variations in the target value can be explained by the model and 40% is not explainable
    R2_train = model.score(features_train, target_train)
    R2_test = model.score(features_test, target_test)

    # If the test error significantly differs from the train error, then there is either overfitting or underfitting

    print('Training set MSE: {}'.format(train_MSE))
    print('Test set MSE: {}'.format(test_MSE))

    print('Training set R2: {}'.format(R2_train))
    print('Test set R2: {}'.format(R2_test))

    # Carry out cross validation of the model.
    # The evaluation method is the root mean square error
    # The method expects a utility function (greater is better) and so the scoring function is the opposite of the the RMSE. Hence the -ve
    tree_rmses = -cross_val_score(model, features_train, target_train,
                                  scoring="neg_root_mean_squared_error", cv=10)

    print(pd.Series(tree_rmses).describe())

    return {'train_MSE': train_MSE, 'test_MSE': test_MSE, 'R2_train': R2_train, 'R2_test': R2_test}


# Decision Tree Model
def DecionTreeRegression(features_train, features_test, target_train, target_test):
    # The DecisionTreeRegressor is passed as the model to the TransformedTreeRegressor together with the StandardScaler
    model = TransformedTargetRegressor(
        DecisionTreeRegressor(), transformer=StandardScaler())
    model.fit(features_train, target_train)

    pred_train = model.predict(features_train)
    pred_test = model.predict(features_test)

    train_MSE = mean_squared_error(target_train, pred_train)
    test_MSE = mean_squared_error(target_test, pred_test)

    R2_train = model.score(features_train, target_train)
    R2_test = model.score(features_test, target_test)

    print('Training set MSE: {}'.format(train_MSE))
    print('Test set MSE: {}'.format(test_MSE))
    print('Training set R2: {}'.format(R2_train))
    print('Test set R2: {}'.format(R2_test))

    from sklearn.model_selection import cross_val_score

    tree_rmses = -cross_val_score(model, features_train, target_train,
                                  scoring="neg_root_mean_squared_error", cv=10)
    print(pd.Series(tree_rmses).describe())
    return {'train_MSE': train_MSE, 'test_MSE': test_MSE, 'R2_train': R2_train, 'R2_test': R2_test}


# RandomForestRegressor
def RandomForestRegression(features_train, features_test, target_train, target_test):
    # RandomForestRegressor is an ensemble method
    # The TransformedTargetRegressor is passed the RandomForestRegressor model
    # The RandomForestRegressor is passed some hyper-parameters such as;
    # n_esimtaors: number of trees in the forest,
    # max_depth: the maximum depth of the tree,
    # criterion: the function to measure the quality of the split

    model = TransformedTargetRegressor(RandomForestRegressor(
        n_estimators=200,  max_depth=8, criterion="squared_error", random_state=18), transformer=StandardScaler())
    model.fit(features_train, target_train)

    pred_train = model.predict(features_train)
    pred_test = model.predict(features_test)

    train_MSE = mean_squared_error(target_train, pred_train)
    test_MSE = mean_squared_error(target_test, pred_test)

    R2_train = model.score(features_train, target_train)
    R2_test = model.score(features_test, target_test)

    print('Training set MSE: {}'.format(train_MSE))
    print('Test set MSE: {}'.format(test_MSE))
    print('Training set R2: {}'.format(R2_train))
    print('Test set R2: {}'.format(R2_test))

    tree_rmses = -cross_val_score(model, features_train, target_train,
                                  scoring="neg_root_mean_squared_error", cv=10)
    print(pd.Series(tree_rmses).describe())

    return {'train_MSE': train_MSE, 'test_MSE': test_MSE, 'R2_train': R2_train, 'R2_test': R2_test}


def GridSearchParams(features_train, target_train):
    # Instatiate the model
    model = RandomForestRegressor()

    param_grid = {'n_estimators': [8, 10, 12, 14, 16, 18, 20]}

    # Define the possible values of the hyperparameter
    grid = {
        'n_estimators': [8, 10, 12, 14, 16, 18, 20, 200, 300, 400, 500],
        'max_features': ['sqrt', 'log2'],
        'max_depth': [4, 5, 6, 7, 8],
        'criterion': ['squared_error', 'absolute_error', 'friedman_mse', 'poisson'],
        'random_state': [18]
    }

    # Deine the model with cv=3 for a 3-fold cross validation
    # GridSearchCV has the best_estimator_ parameter that returns the  estimator
    # which gave highest score (or smallest loss if specified)

    grid_search = GridSearchCV(
        model, grid, cv=3, scoring='neg_root_mean_squared_error')
    grid_search.fit(features_train, target_train)

    # Get the best param combination
    print(grid_search.best_estimator_)

    return {'train_MSE': train_MSE, 'test_MSE': test_MSE, 'R2_train': R2_train, 'R2_test': R2_test}

### Linear Regression


#### Goalkeepers


In [82]:
gk_lin_reg = Linear_regression(goalkeepers_splits['feature_train'], goalkeepers_splits['features_test'],
                               goalkeepers_splits['target_train'], goalkeepers_splits['target_test'])

# Store the model evaluation details in a DataFrame
evaluation_stats = pd.DataFrame({"gk_lin_reg": [gk_lin_reg['train_MSE'], gk_lin_reg['test_MSE'],
                                gk_lin_reg['R2_train'], gk_lin_reg['R2_test']]}, index=(['train_MSE', 'test_MSE', 'R2_train', 'R2_test']))

Training set MSE: 1.3679500648313132
Test set MSE: 1.8240359929326215
Training set R2: 0.5891590489075507
Test set R2: 0.4940646376660027
count    10.000000
mean      1.175864
std       0.164150
min       1.005387
25%       1.119668
50%       1.137787
75%       1.172158
max       1.618587
dtype: float64


#### Defenders


In [88]:
def_lin_reg = Linear_regression(defenders_splits['feature_train'], defenders_splits['features_test'],
                                defenders_splits['target_train'], defenders_splits['target_test'])

# Store the model evaluation details in the evaluation_stats DataFrame
evaluation_stats = evaluation_stats.assign(def_lin_reg=[
                                           def_lin_reg['train_MSE'], def_lin_reg['test_MSE'], def_lin_reg['R2_train'], def_lin_reg['R2_test']])
evaluation_stats

Training set MSE: 2.3555511130733553
Test set MSE: 2.242464061936939
Training set R2: 0.5085692276325189
Test set R2: 0.5214858041329767
count    10.000000
mean      1.536047
std       0.139393
min       1.364620
25%       1.451274
50%       1.478606
75%       1.665341
max       1.744419
dtype: float64


,gk_lin_reg,def_lin_reg
train_MSE,1.367950,2.355551
test_MSE,1.824036,2.242464
R2_train,0.589159,0.508569
R2_test,0.494065,0.521486


#### Midfielders


In [89]:
mid_lin_reg = Linear_regression(midfielders_splits['feature_train'], midfielders_splits['features_test'],
                                midfielders_splits['target_train'], midfielders_splits['target_test'])

# Store the model evaluation details in the evaluation_stats DataFrame
evaluation_stats = evaluation_stats.assign(mid_lin_reg=[
                                           mid_lin_reg['train_MSE'], mid_lin_reg['test_MSE'], mid_lin_reg['R2_train'], mid_lin_reg['R2_test']])
evaluation_stats

Training set MSE: 1.4543649299444035
Test set MSE: 1.310604401914415
Training set R2: 0.7531120839723451
Test set R2: 0.706409711574788
count    10.000000
mean      1.206041
std       0.095865
min       1.098467
25%       1.127169
50%       1.198590
75%       1.241352
max       1.408256
dtype: float64


,gk_lin_reg,def_lin_reg,mid_lin_reg
train_MSE,1.367950,2.355551,1.454365
test_MSE,1.824036,2.242464,1.310604
R2_train,0.589159,0.508569,0.753112
R2_test,0.494065,0.521486,0.706410


#### Forwards


In [91]:
for_lin_reg = Linear_regression(forwards_splits['feature_train'], forwards_splits['features_test'],
                                forwards_splits['target_train'], forwards_splits['target_test'])

# Store the model evaluation details in the evaluation_stats DataFrame
evaluation_stats = evaluation_stats.assign(for_lin_reg=[
                                           for_lin_reg['train_MSE'], for_lin_reg['test_MSE'], for_lin_reg['R2_train'], for_lin_reg['R2_test']])
evaluation_stats

Training set MSE: 0.9787146939483393
Test set MSE: 1.1061503964732393
Training set R2: 0.8570883045385085
Test set R2: 0.8040186118528224
count    10.000000
mean      0.995046
std       0.134777
min       0.757743
25%       0.991994
50%       1.019414
75%       1.054394
max       1.149812
dtype: float64


,gk_lin_reg,def_lin_reg,mid_lin_reg,for_lin_reg
train_MSE,1.367950,2.355551,1.454365,0.978715
test_MSE,1.824036,2.242464,1.310604,1.106150
R2_train,0.589159,0.508569,0.753112,0.857088
R2_test,0.494065,0.521486,0.706410,0.804019


### DecionTreeRegressor


#### Goalkeepers


In [37]:
DecionTreeRegression(goalkeepers_splits['feature_train'], goalkeepers_splits['features_test'],
                     goalkeepers_splits['target_train'], goalkeepers_splits['target_test'])

Training set MSE: 0.007368425258833918
Test set MSE: 2.253328305153419
Training set R2: 0.9977870165591416
Test set R2: 0.3749912408842696
count    10.000000
mean      1.468319
std       0.143604
min       1.278020
25%       1.336350
50%       1.509847
75%       1.568238
max       1.701541
dtype: float64


#### Defenders


In [38]:
DecionTreeRegression(defenders_splits['feature_train'], defenders_splits['features_test'],
                     defenders_splits['target_train'], defenders_splits['target_test'])

Training set MSE: 0.005552267620610064
Test set MSE: 3.7669020089498253
Training set R2: 0.9988416489245154
Test set R2: 0.1961895326137052
count    10.000000
mean      2.018183
std       0.171052
min       1.795607
25%       1.925892
50%       1.968639
75%       2.138815
max       2.328743
dtype: float64


#### Midfielders


In [39]:
DecionTreeRegression(midfielders_splits['feature_train'], midfielders_splits['features_test'],
                     midfielders_splits['target_train'], midfielders_splits['target_test'])

Training set MSE: 0.0044166453997284884
Test set MSE: 2.168530058806828
Training set R2: 0.9992502456872266
Test set R2: 0.5142246092765576
count    10.000000
mean      1.624650
std       0.090965
min       1.478617
25%       1.561261
50%       1.624745
75%       1.676276
max       1.790597
dtype: float64


#### Forwards


In [40]:
DecionTreeRegression(forwards_splits['feature_train'], forwards_splits['features_test'],
                     forwards_splits['target_train'], forwards_splits['target_test'])

Training set MSE: 0.0050067457005697185
Test set MSE: 1.8983484620706783
Training set R2: 0.9992689161394661
Test set R2: 0.6636614984997912
count    10.000000
mean      1.424385
std       0.178749
min       1.108509
25%       1.309020
50%       1.475370
75%       1.576077
max       1.609017
dtype: float64


### RandomForestRegressor


#### Goalkeeprs


In [41]:
RandomForestRegression(goalkeepers_splits['feature_train'], goalkeepers_splits['features_test'],
                       goalkeepers_splits['target_train'], goalkeepers_splits['target_test'])

Training set MSE: 0.5418093964511264
Test set MSE: 1.1670709929499417
Training set R2: 0.8372765984142441
Test set R2: 0.676287919769444


count    10.000000
mean      1.111343
std       0.119699
min       0.949683
25%       0.995994
50%       1.152811
75%       1.192800
max       1.301595
dtype: float64


#### Defenders


In [42]:
RandomForestRegression(defenders_splits['feature_train'], defenders_splits['features_test'],
                       defenders_splits['target_train'], defenders_splits['target_test'])

Training set MSE: 1.2651863472213685
Test set MSE: 1.9310999606410764
Training set R2: 0.7360483920925953
Test set R2: 0.5879270662617242
count    10.000000
mean      1.456177
std       0.140725
min       1.255310
25%       1.359548
50%       1.417128
75%       1.562341
max       1.669058
dtype: float64


#### Midfielders


In [43]:
RandomForestRegression(midfielders_splits['feature_train'], midfielders_splits['features_test'],
                       midfielders_splits['target_train'], midfielders_splits['target_test'])

Training set MSE: 0.6845720225875751
Test set MSE: 1.1603304192826107
Training set R2: 0.88378944201168
Test set R2: 0.7400727923940128
count    10.000000
mean      1.150480
std       0.099538
min       1.028633
25%       1.082559
50%       1.132939
75%       1.222564
max       1.333819
dtype: float64


#### Forwards


In [44]:
RandomForestRegression(forwards_splits['feature_train'], forwards_splits['features_test'],
                       forwards_splits['target_train'], forwards_splits['target_test'])

Training set MSE: 0.29956071588293404
Test set MSE: 0.8804356065767887
Training set R2: 0.9562582128732727
Test set R2: 0.8440094648058143
count    10.000000
mean      1.034057
std       0.066542
min       0.945428
25%       1.002534
50%       1.025271
75%       1.039612
max       1.159691
dtype: float64


#### Fine Tunning the RandomForestRegressor

From the above, we see that RandomForestRegressor performs the best MSE and they do not differ that much on the test and the train sets.
In order to improve its performance we carry out Cross validation.


#### GridSearchCV

At this stage we are doing hyperparameter tuning i.e. finding the best combination of hyperparameters to be passed to the RandomForestRegressor.


#### Goalkeepers


In [45]:
GridSearchParams(
    goalkeepers_splits['feature_train'], goalkeepers_splits['target_train'])

RandomForestRegressor(criterion='friedman_mse', max_depth=8,
                      max_features='sqrt', n_estimators=20, random_state=18)


c:\Users\Ilyas\anaconda3\envs\FPL\Lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
220 fits failed out of a total of 1320.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
220 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Ilyas\anaconda3\envs\FPL\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Ilyas\anaconda3\envs\FPL\Lib\site-packages\sklearn\ensemble\_forest.py", line 359, in fit
    raise ValueError(
ValueError: Some value(s) of y are negative which is not allowed for Poisson regression.

  warnings.warn(some_fits_failed_message, FitFailedWarning)
c:\Users\

##### RandomForestRegressor using the best hyperparameters from the GridSearchCV


In [46]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.compose import TransformedTargetRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score


# RandomForestRegressor is an ensemble method
# The TransformedTargetRegressor is passed the RandomForestRegressor model
# The RandomForestRegressor is passed some hyper-parameters such as;
# n_esimtaors: number of trees in the forest,
# max_depth: the maximum depth of the tree,
# criterion: the function to measure the quality of the split

model = TransformedTargetRegressor(RandomForestRegressor(criterion='friedman_mse', max_depth=8,
                                                         max_features='sqrt', n_estimators=300, random_state=18), transformer=StandardScaler())
model.fit(goalkeepers_splits['feature_train'],
          goalkeepers_splits['target_train'])

pred_train = model.predict(goalkeepers_splits['feature_train'])
pred_test = model.predict(goalkeepers_splits['features_test'])

train_MSE = mean_squared_error(goalkeepers_splits['target_train'], pred_train)
test_MSE = mean_squared_error(goalkeepers_splits['target_test'], pred_test)

R2_train = model.score(
    goalkeepers_splits['feature_train'], goalkeepers_splits['target_train'])
R2_test = model.score(
    goalkeepers_splits['features_test'], goalkeepers_splits['target_test'])

print('Training set MSE: {}'.format(train_MSE))
print('Test set MSE: {}'.format(test_MSE))
print('Training set R2: {}'.format(R2_train))
print('Test set R2: {}'.format(R2_test))

tree_rmses = -cross_val_score(model, goalkeepers_splits['feature_train'], goalkeepers_splits['target_train'],
                              scoring="neg_root_mean_squared_error", cv=10)

pd.Series(tree_rmses).describe()

Training set MSE: 0.710492022581101
Test set MSE: 1.1396069741953987
Training set R2: 0.7866155894098279
Test set R2: 0.6839056522777743


count    10.000000
mean      1.138626
std       0.129259
min       0.994879
25%       1.030045
50%       1.093788
75%       1.244792
max       1.356697
dtype: float64

#### Defenders


In [47]:
GridSearchParams(defenders_splits['feature_train'],
                 defenders_splits['target_train'])

c:\Users\Ilyas\anaconda3\envs\FPL\Lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
330 fits failed out of a total of 1320.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
330 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Ilyas\anaconda3\envs\FPL\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Ilyas\anaconda3\envs\FPL\Lib\site-packages\sklearn\ensemble\_forest.py", line 359, in fit
    raise ValueError(
ValueError: Some value(s) of y are negative which is not allowed for Poisson regression.

  warnings.warn(some_fits_failed_message, FitFailedWarning)
c:\Users\

RandomForestRegressor(criterion='friedman_mse', max_depth=8,
                      max_features='sqrt', n_estimators=300, random_state=18)


##### RandomForestRegressor using the best hyperparameters from the GridSearchCV


In [48]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.compose import TransformedTargetRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score


# RandomForestRegressor is an ensemble method
# The TransformedTargetRegressor is passed the RandomForestRegressor model
# The RandomForestRegressor is passed some hyper-parameters such as;
# n_esimtaors: number of trees in the forest,
# max_depth: the maximum depth of the tree,
# criterion: the function to measure the quality of the split

model = TransformedTargetRegressor(RandomForestRegressor(criterion='friedman_mse', max_depth=8,
                                                         max_features='sqrt', n_estimators=200, random_state=18), transformer=StandardScaler())
model.fit(defenders_splits['feature_train'], defenders_splits['target_train'])

pred_train = model.predict(defenders_splits['feature_train'])
pred_test = model.predict(defenders_splits['features_test'])

train_MSE = mean_squared_error(defenders_splits['target_train'], pred_train)
test_MSE = mean_squared_error(defenders_splits['target_test'], pred_test)

R2_train = model.score(
    defenders_splits['feature_train'], defenders_splits['target_train'])
R2_test = model.score(
    defenders_splits['features_test'], defenders_splits['target_test'])

print('Training set MSE: {}'.format(train_MSE))
print('Test set MSE: {}'.format(test_MSE))
print('Training set R2: {}'.format(R2_train))
print('Test set R2: {}'.format(R2_test))

tree_rmses = -cross_val_score(model, defenders_splits['feature_train'], defenders_splits['target_train'],
                              scoring="neg_root_mean_squared_error", cv=10)

pd.Series(tree_rmses).describe()

Training set MSE: 1.4554544608019722
Test set MSE: 2.0325912002563693
Training set R2: 0.6963533901480936
Test set R2: 0.5662700864525981


count    10.000000
mean      1.469367
std       0.150307
min       1.267562
25%       1.360834
50%       1.438250
75%       1.598995
max       1.684292
dtype: float64

#### Midfielders


In [49]:
GridSearchParams(
    midfielders_splits['feature_train'], midfielders_splits['target_train'])

c:\Users\Ilyas\anaconda3\envs\FPL\Lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
330 fits failed out of a total of 1320.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
330 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Ilyas\anaconda3\envs\FPL\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Ilyas\anaconda3\envs\FPL\Lib\site-packages\sklearn\ensemble\_forest.py", line 359, in fit
    raise ValueError(
ValueError: Some value(s) of y are negative which is not allowed for Poisson regression.

  warnings.warn(some_fits_failed_message, FitFailedWarning)
c:\Users\

RandomForestRegressor(criterion='friedman_mse', max_depth=8,
                      max_features='sqrt', n_estimators=400, random_state=18)


##### RandomForestRegressor using the best hyperparameters from the GridSearchCV


In [50]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.compose import TransformedTargetRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score


# RandomForestRegressor is an ensemble method
# The TransformedTargetRegressor is passed the RandomForestRegressor model
# The RandomForestRegressor is passed some hyper-parameters such as;
# n_esimtaors: number of trees in the forest,
# max_depth: the maximum depth of the tree,
# criterion: the function to measure the quality of the split

model = TransformedTargetRegressor(RandomForestRegressor(criterion='friedman_mse', max_depth=8,
                                                         max_features='sqrt', n_estimators=300, random_state=18), transformer=StandardScaler())
model.fit(midfielders_splits['feature_train'],
          midfielders_splits['target_train'])

pred_train = model.predict(midfielders_splits['feature_train'])
pred_test = model.predict(midfielders_splits['features_test'])

train_MSE = mean_squared_error(midfielders_splits['target_train'], pred_train)
test_MSE = mean_squared_error(midfielders_splits['target_test'], pred_test)

R2_train = model.score(
    midfielders_splits['feature_train'], midfielders_splits['target_train'])
R2_test = model.score(
    midfielders_splits['features_test'], midfielders_splits['target_test'])

print('Training set MSE: {}'.format(train_MSE))
print('Test set MSE: {}'.format(test_MSE))
print('Training set R2: {}'.format(R2_train))
print('Test set R2: {}'.format(R2_test))

tree_rmses = -cross_val_score(model, midfielders_splits['feature_train'], midfielders_splits['target_train'],
                              scoring="neg_root_mean_squared_error", cv=10)

pd.Series(tree_rmses).describe()

Training set MSE: 0.8110497936355873
Test set MSE: 1.1677454695921783
Training set R2: 0.862319016896946
Test set R2: 0.7384117368108838


count    10.000000
mean      1.179908
std       0.105541
min       1.060674
25%       1.097547
50%       1.166847
75%       1.243059
max       1.389151
dtype: float64

#### Forwards


In [51]:
GridSearchParams(forwards_splits['feature_train'],
                 forwards_splits['target_train'])

c:\Users\Ilyas\anaconda3\envs\FPL\Lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
330 fits failed out of a total of 1320.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
330 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Ilyas\anaconda3\envs\FPL\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Ilyas\anaconda3\envs\FPL\Lib\site-packages\sklearn\ensemble\_forest.py", line 359, in fit
    raise ValueError(
ValueError: Some value(s) of y are negative which is not allowed for Poisson regression.

  warnings.warn(some_fits_failed_message, FitFailedWarning)
c:\Users\

RandomForestRegressor(criterion='friedman_mse', max_depth=8,
                      max_features='sqrt', n_estimators=300, random_state=18)


##### RandomForestRegressor using the best hyperparameters from the GridSearchCV


In [52]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.compose import TransformedTargetRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score


# RandomForestRegressor is an ensemble method
# The TransformedTargetRegressor is passed the RandomForestRegressor model
# The RandomForestRegressor is passed some hyper-parameters such as;
# n_esimtaors: number of trees in the forest,
# max_depth: the maximum depth of the tree,
# criterion: the function to measure the quality of the split

model = TransformedTargetRegressor(RandomForestRegressor(criterion='friedman_mse', max_depth=8,
                                                         max_features='sqrt', n_estimators=500, random_state=18), transformer=StandardScaler())
model.fit(forwards_splits['feature_train'], forwards_splits['target_train'])

pred_train = model.predict(forwards_splits['feature_train'])
pred_test = model.predict(forwards_splits['features_test'])

train_MSE = mean_squared_error(forwards_splits['target_train'], pred_train)
test_MSE = mean_squared_error(forwards_splits['target_test'], pred_test)

R2_train = model.score(
    forwards_splits['feature_train'], forwards_splits['target_train'])
R2_test = model.score(
    forwards_splits['features_test'], forwards_splits['target_test'])

print('Training set MSE: {}'.format(train_MSE))
print('Test set MSE: {}'.format(test_MSE))
print('Training set R2: {}'.format(R2_train))
print('Test set R2: {}'.format(R2_test))

tree_rmses = -cross_val_score(model, forwards_splits['feature_train'], forwards_splits['target_train'],
                              scoring="neg_root_mean_squared_error", cv=10)

pd.Series(tree_rmses).describe()

Training set MSE: 0.4248974211561101
Test set MSE: 1.0099176727007482
Training set R2: 0.9379565758743575
Test set R2: 0.8210685743626654


count    10.000000
mean      1.119247
std       0.118388
min       0.863989
25%       1.076667
50%       1.131864
75%       1.176880
max       1.280818
dtype: float64

### XGBoost


In [ ]:
%pip install xgboost

In [55]:
from xgboost import XGBRegressor as xgb
from sklearn.model_selection import GridSearchCV
import numpy as np

# =========================================================================
# XGBoost regression:
# Parameters:
# n_estimators  "Number of gradient boosted trees. Equivalent to number
#                of boosting rounds."
# learning_rate "Boosting learning rate (also known as “eta”)"
# max_depth     "Maximum depth of a tree. Increasing this value will make
#                the model more complex and more likely to overfit."
# =========================================================================
regressor = xgb(eval_metric='rmsle')

# =========================================================================
# exhaustively search for the optimal hyperparameters
# =========================================================================
# set up our search grid
param_grid = {"max_depth":    [4, 5, 6],
              "n_estimators": np.arange(100, 1000, 50),
              "learning_rate": np.arange(0.01, .2, 0.005)}

# try out every combination of the above values
search = GridSearchCV(regressor, param_grid, cv=5).fit(
    goalkeepers_splits['feature_train'], goalkeepers_splits['target_train'])

print("The best hyperparameters are ", search.best_params_)

The best hyperparameters are  {'learning_rate': 0.019999999999999997, 'max_depth': 4, 'n_estimators': 150}


In [56]:
from xgboost import XGBRegressor as xgb
from sklearn.model_selection import GridSearchCV
import numpy as np

# =========================================================================
# XGBoost regression:
# Parameters:
# n_estimators  "Number of gradient boosted trees. Equivalent to number
#                of boosting rounds."
# learning_rate "Boosting learning rate (also known as “eta”)"
# max_depth     "Maximum depth of a tree. Increasing this value will make
#                the model more complex and more likely to overfit."
# =========================================================================
regressor = xgb(eval_metric='rmsle')

# =========================================================================
# exhaustively search for the optimal hyperparameters
# =========================================================================
# set up our search grid
param_grid = {"max_depth":    [4, 5, 6],
              "n_estimators": [100, 200, 300, 400, 500, 600, 700],
              "learning_rate": np.arange(0.01, .2, 0.005)}

# try out every combination of the above values
goalkeepers_search = GridSearchCV(regressor, param_grid, cv=5).fit(
    goalkeepers_splits['feature_train'], goalkeepers_splits['target_train'])
defenders_search = GridSearchCV(regressor, param_grid, cv=5).fit(
    defenders_splits['feature_train'], defenders_splits['target_train'])
midfielders_search = GridSearchCV(regressor, param_grid, cv=5).fit(
    midfielders_splits['feature_train'], midfielders_splits['target_train'])
forwards_search = GridSearchCV(regressor, param_grid, cv=5).fit(
    forwards_splits['feature_train'], forwards_splits['target_train'])

print("The best hyperparameters are ", goalkeepers_search.best_params_)
print("The best defenders hyperparameters are ", defenders_search.best_params_)
print("The best midfielders hyperparameters are ",
      midfielders_search.best_params_)
print("The best midfielder hyperparameters are ", forwards_search.best_params_)

The best hyperparameters are  {'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 300}
The best defenders hyperparameters are  {'learning_rate': 0.03, 'max_depth': 4, 'n_estimators': 300}
The best midfielders hyperparameters are  {'learning_rate': 0.039999999999999994, 'max_depth': 4, 'n_estimators': 100}
The best midfielder hyperparameters are  {'learning_rate': 0.024999999999999998, 'max_depth': 4, 'n_estimators': 300}


In [57]:
from xgboost import XGBRegressor as xgb
from sklearn.model_selection import GridSearchCV
import numpy as np

# =========================================================================
# XGBoost regression:
# Parameters:
# n_estimators  "Number of gradient boosted trees. Equivalent to number
#                of boosting rounds."
# learning_rate "Boosting learning rate (also known as “eta”)"
# max_depth     "Maximum depth of a tree. Increasing this value will make
#                the model more complex and more likely to overfit."
# =========================================================================
regressor = xgb(eval_metric='rmsle')

# =========================================================================
# exhaustively search for the optimal hyperparameters
# =========================================================================
# set up our search grid
param_grid = {"max_depth":    [4, 5, 6],
              "n_estimators": np.arange(100, 1000, 50),
              "learning_rate": np.arange(0.01, .2, 0.005)}

# try out every combination of the above values
goalkeepers_search = GridSearchCV(regressor, param_grid, cv=5).fit(
    goalkeepers_splits['feature_train'], goalkeepers_splits['target_train'])
defenders_search = GridSearchCV(regressor, param_grid, cv=5).fit(
    defenders_splits['feature_train'], defenders_splits['target_train'])
midfielders_search = GridSearchCV(regressor, param_grid, cv=5).fit(
    midfielders_splits['feature_train'], midfielders_splits['target_train'])
forwards_search = GridSearchCV(regressor, param_grid, cv=5).fit(
    forwards_splits['feature_train'], forwards_splits['target_train'])

print("The best hyperparameters are ", goalkeepers_search.best_params_)
print("The best defenders hyperparameters are ", defenders_search.best_params_)
print("The best midfielders hyperparameters are ",
      midfielders_search.best_params_)
print("The best midfielder hyperparameters are ", forwards_search.best_params_)

The best hyperparameters are  {'learning_rate': 0.019999999999999997, 'max_depth': 4, 'n_estimators': 150}
The best defenders hyperparameters are  {'learning_rate': 0.03, 'max_depth': 4, 'n_estimators': 350}
The best midfielders hyperparameters are  {'learning_rate': 0.039999999999999994, 'max_depth': 4, 'n_estimators': 100}
The best midfielder hyperparameters are  {'learning_rate': 0.024999999999999998, 'max_depth': 4, 'n_estimators': 250}


#### xgboost using the best hyperparameters from GridSearchCV


##### GoalKeepers


In [64]:
regressor = xgb(learning_rate=goalkeepers_search.best_params_["learning_rate"],
                n_estimators=goalkeepers_search.best_params_["n_estimators"],
                max_depth=goalkeepers_search.best_params_["max_depth"],
                eval_metric='rmsle')

regressor.fit(goalkeepers_splits['feature_train'],
              goalkeepers_splits['target_train'])

# =========================================================================
# To use early_stopping_rounds:
# "Validation metric needs to improve at least once in every
# early_stopping_rounds round(s) to continue training."
# =========================================================================
# first perform a test/train split
# from sklearn.model_selection import train_test_split

# X_train,X_test,y_train,y_test = train_test_split(X_train,y_train, test_size = 0.2)
# regressor.fit(X_train, y_train, early_stopping_rounds=6, eval_set=[(X_test, y_test)], verbose=False)

# =========================================================================
# use the model to predict the prices for the test data
# =========================================================================
# predictions = regressor.predict(goalkeepers_splits['feature_test'])


pred_train = regressor.predict(goalkeepers_splits['feature_train'])
pred_test = regressor.predict(goalkeepers_splits['features_test'])

train_MSE = mean_squared_error(goalkeepers_splits['target_train'], pred_train)
test_MSE = mean_squared_error(goalkeepers_splits['target_test'], pred_test)

R2_train = model.score(
    goalkeepers_splits['feature_train'], goalkeepers_splits['target_train'])
R2_test = model.score(
    goalkeepers_splits['features_test'], goalkeepers_splits['target_test'])

print('Training set MSE: {}'.format(train_MSE))
print('Test set MSE: {}'.format(test_MSE))
print('Training set R2: {}'.format(R2_train))
print('Test set R2: {}'.format(R2_test))

tree_rmses = -cross_val_score(model, goalkeepers_splits['feature_train'], goalkeepers_splits['target_train'],
                              scoring="neg_root_mean_squared_error", cv=10)
pd.Series(tree_rmses).describe()

Training set MSE: 0.8235964074241952
Test set MSE: 1.1785011992155054
Training set R2: 0.4745668201526443
Test set R2: 0.45910408469637176


count    10.000000
mean      1.137168
std       0.129813
min       0.996080
25%       1.033505
50%       1.090561
75%       1.237824
max       1.362908
dtype: float64

##### Defenders


In [65]:
regressor = xgb(learning_rate=defenders_search.best_params_["learning_rate"],
                n_estimators=defenders_search.best_params_["n_estimators"],
                max_depth=defenders_search.best_params_["max_depth"],
                eval_metric='rmsle')

regressor.fit(defenders_splits['feature_train'],
              defenders_splits['target_train'])

# =========================================================================
# To use early_stopping_rounds:
# "Validation metric needs to improve at least once in every
# early_stopping_rounds round(s) to continue training."
# =========================================================================
# first perform a test/train split
# from sklearn.model_selection import train_test_split

# X_train,X_test,y_train,y_test = train_test_split(X_train,y_train, test_size = 0.2)
# regressor.fit(X_train, y_train, early_stopping_rounds=6, eval_set=[(X_test, y_test)], verbose=False)

# =========================================================================
# use the model to predict the prices for the test data
# =========================================================================
# predictions = regressor.predict(defenders_splits['feature_test'])


pred_train = regressor.predict(defenders_splits['feature_train'])
pred_test = regressor.predict(defenders_splits['features_test'])

train_MSE = mean_squared_error(defenders_splits['target_train'], pred_train)
test_MSE = mean_squared_error(defenders_splits['target_test'], pred_test)

R2_train = model.score(
    defenders_splits['feature_train'], defenders_splits['target_train'])
R2_test = model.score(
    defenders_splits['features_test'], defenders_splits['target_test'])

print('Training set MSE: {}'.format(train_MSE))
print('Test set MSE: {}'.format(test_MSE))
print('Training set R2: {}'.format(R2_train))
print('Test set R2: {}'.format(R2_test))

tree_rmses = -cross_val_score(model, defenders_splits['feature_train'], defenders_splits['target_train'],
                              scoring="neg_root_mean_squared_error", cv=10)
pd.Series(tree_rmses).describe()

Training set MSE: 1.3335908479695675
Test set MSE: 1.9970324779667072
Training set R2: 0.4899905690420201
Test set R2: 0.4801206460183943


count    10.000000
mean      1.469514
std       0.152986
min       1.260577
25%       1.358206
50%       1.434897
75%       1.602652
max       1.691610
dtype: float64

##### Midfielders


In [66]:
regressor = xgb(learning_rate=midfielders_search.best_params_["learning_rate"],
                n_estimators=midfielders_search.best_params_["n_estimators"],
                max_depth=midfielders_search.best_params_["max_depth"],
                eval_metric='rmsle')

regressor.fit(midfielders_splits['feature_train'],
              midfielders_splits['target_train'])

# =========================================================================
# To use early_stopping_rounds:
# "Validation metric needs to improve at least once in every
# early_stopping_rounds round(s) to continue training."
# =========================================================================
# first perform a test/train split
# from sklearn.model_selection import train_test_split

# X_train,X_test,y_train,y_test = train_test_split(X_train,y_train, test_size = 0.2)
# regressor.fit(X_train, y_train, early_stopping_rounds=6, eval_set=[(X_test, y_test)], verbose=False)

# =========================================================================
# use the model to predict the prices for the test data
# =========================================================================
# predictions = regressor.predict(midfielders_splits['feature_test'])


pred_train = regressor.predict(midfielders_splits['feature_train'])
pred_test = regressor.predict(midfielders_splits['features_test'])

train_MSE = mean_squared_error(midfielders_splits['target_train'], pred_train)
test_MSE = mean_squared_error(midfielders_splits['target_test'], pred_test)

R2_train = model.score(
    midfielders_splits['feature_train'], midfielders_splits['target_train'])
R2_test = model.score(
    midfielders_splits['features_test'], midfielders_splits['target_test'])

print('Training set MSE: {}'.format(train_MSE))
print('Test set MSE: {}'.format(test_MSE))
print('Training set R2: {}'.format(R2_train))
print('Test set R2: {}'.format(R2_test))

tree_rmses = -cross_val_score(model, midfielders_splits['feature_train'], midfielders_splits['target_train'],
                              scoring="neg_root_mean_squared_error", cv=10)
pd.Series(tree_rmses).describe()

Training set MSE: 0.9659007876594823
Test set MSE: 1.1527634506494209
Training set R2: 0.7444468790927337
Test set R2: 0.7227527515757084


count    10.000000
mean      1.180358
std       0.105118
min       1.063850
25%       1.096319
50%       1.166338
75%       1.238633
max       1.388128
dtype: float64

##### Forwards


In [67]:
regressor = xgb(learning_rate=forwards_search.best_params_["learning_rate"],
                n_estimators=forwards_search.best_params_["n_estimators"],
                max_depth=forwards_search.best_params_["max_depth"],
                eval_metric='rmsle')

regressor.fit(forwards_splits['feature_train'],
              forwards_splits['target_train'])

# =========================================================================
# To use early_stopping_rounds:
# "Validation metric needs to improve at least once in every
# early_stopping_rounds round(s) to continue training."
# =========================================================================
# first perform a test/train split
# from sklearn.model_selection import train_test_split

# X_train,X_test,y_train,y_test = train_test_split(X_train,y_train, test_size = 0.2)
# regressor.fit(X_train, y_train, early_stopping_rounds=6, eval_set=[(X_test, y_test)], verbose=False)

# =========================================================================
# use the model to predict the prices for the test data
# =========================================================================
# predictions = regressor.predict(forwards_splits['feature_test'])


pred_train = regressor.predict(forwards_splits['feature_train'])
pred_test = regressor.predict(forwards_splits['features_test'])

train_MSE = mean_squared_error(forwards_splits['target_train'], pred_train)
test_MSE = mean_squared_error(forwards_splits['target_test'], pred_test)

R2_train = model.score(
    forwards_splits['feature_train'], forwards_splits['target_train'])
R2_test = model.score(
    forwards_splits['features_test'], forwards_splits['target_test'])

print('Training set MSE: {}'.format(train_MSE))
print('Test set MSE: {}'.format(test_MSE))
print('Training set R2: {}'.format(R2_train))
print('Test set R2: {}'.format(R2_test))

tree_rmses = -cross_val_score(model, forwards_splits['feature_train'], forwards_splits['target_train'],
                              scoring="neg_root_mean_squared_error", cv=10)
pd.Series(tree_rmses).describe()

Training set MSE: 0.3860503542746733
Test set MSE: 0.9080504130026612
Training set R2: 0.9379565758743575
Test set R2: 0.8210685743626654


count    10.000000
mean      1.119247
std       0.118388
min       0.863989
25%       1.076667
50%       1.131864
75%       1.176880
max       1.280818
dtype: float64